# Preparation

In [ ]:
%cd /home/dvc-2-iris-demo

In [ ]:
from sklearn.model_selection import train_test_split

from src.data.dataset import get_dataset
from src.evaluate.evaluate import evaluate
from src.features.features import extract_features
from src.report.visualize import plot_confusion_matrix
from src.train.train import get_supported_estimator, train
from src.transforms.trainsforms import transform_target_values_to_labels

# Load dataset

In [ ]:
dataset = get_dataset('data/raw/iris.csv')
dataset.shape

In [ ]:
dataset.head()

In [ ]:
# feature names
feature_names = dataset.columns.tolist()[:4]
feature_names 

In [ ]:
# iris species
species = dataset['species'].unique().tolist()
species

# Features engineering

In [ ]:
dataset = extract_features(dataset)
dataset.head()

# Split dataset

In [ ]:
test_size = 0.2
random_state = 42

## transform targets (species) to numerics

In [ ]:
dataset = transform_target_values_to_labels(dataset, 'species')
dataset.head()

## Split in train/test

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size=test_size, random_state=42)
train_dataset.shape, test_dataset.shape

# Train

In [ ]:
# supported classifiers
get_supported_estimator()

In [ ]:
estimator_name = 'logreg'
param_grid = { 
        'C': [0.001,0.01],
        'max_iter': [5000,6000],
        'solver': ['lbfgs', 'sag'],
        'multi_class': ['multinomial']
    }

scoring = 'f1_macro'
cv = 3

In [ ]:
model = train(
    df=dataset,
    target_column='species',
    estimator_name=estimator_name,
    param_grid=param_grid,
    cv=cv
)

In [ ]:
model.best_estimator_


# Evaluate

In [ ]:
f1, cm = evaluate(
    df=dataset,
    target_column='species',
    clf=model
)

In [ ]:
f1

In [ ]:
plot_confusion_matrix(cm, species, normalize=False)